# Tarea 6

https://medium.com/@suci/running-pyspark-on-jupyter-notebook-with-docker-602b18ac4494

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[3]").getOrCreate()

## Ejercicio 1

## Ejercicio 2

In [68]:
flights = spark.read.csv('/home/jovyan/flights/flights.csv', header =True)
airports = spark.read.csv('/home/jovyan/flights/airports.csv', header =True)
airlines = spark.read.csv('/home/jovyan/flights/airlines.csv', header =True)

In [69]:
flights.printSchema() 

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

### Ejercicio 2a

2a. ¿Qué aerolíneas (nombres) llegan al aeropuerto "Honolulu International Airport"?

In [129]:
dest = flights.select("AIRLINE").where("DESTINATION_AIRPORT = 'HNL'").distinct()
aerolinea = airlines.select("IATA_CODE", 
                          airlines.AIRLINE.alias("nombre"))
ej_2a = dest.join(aerolinea, dest.AIRLINE == aerolinea.IATA_CODE, 'left')
ej_2a = ej_2a.select("AIRLINE", "nombre")

ej_2a.show()
ej_2a.repartition(1).write.csv('2a')

+-------+--------------------+
|AIRLINE|              nombre|
+-------+--------------------+
|     UA|United Air Lines ...|
|     AA|American Airlines...|
|     DL|Delta Air Lines Inc.|
|     US|     US Airways Inc.|
|     HA|Hawaiian Airlines...|
|     AS|Alaska Airlines Inc.|
|     VX|      Virgin America|
+-------+--------------------+



### Ejercicio 2b

2b. ¿En qué horario (hora del día, no importan los minutos) hay salidas del aeropuerto de San Francisco ("SFO") a "Honolulu International Airport"

In [130]:
horas = flights.withColumn("HORAS", floor(flights.SCHEDULED_DEPARTURE / 100))
ej_2b = horas.select("HORAS").where("destination_airport = 'HNL' AND origin_airport = 'SFO'")\
.distinct()

ej_2b.show()
ej_2b.repartition(1).write.csv('2b')

+-----+
|HORAS|
+-----+
|   19|
|    7|
|    6|
|    9|
|   17|
|   10|
|   12|
|    8|
|   11|
|   13|
|   18|
|   14|
|   15|
|   16|
+-----+



### Ejercicio 2c

2c. ¿Qué día de la semana y en qué aerolínea nos conviene viajar a "Honolulu International Airport" para tener el menor retraso posible? 

In [131]:
retraso = flights.select(flights.DAY_OF_WEEK, 
                         flights.ARRIVAL_DELAY.cast("float").alias("ARRIVAL_DELAY"),
                        flights.DESTINATION_AIRPORT,
                        flights.AIRLINE)
retraso = retraso.where("DESTINATION_AIRPORT = 'HNL'")
retraso = retraso.groupBy("DAY_OF_WEEK", "AIRLINE").avg("ARRIVAL_DELAY")
retraso = retraso.sort(retraso["avg(ARRIVAL_DELAY)"]).limit(3)

aerolinea = airlines.select("IATA_CODE", 
                          airlines.AIRLINE.alias("nombre"))
ej_2c = retraso.join(aerolinea, retraso.AIRLINE == aerolinea.IATA_CODE, 'left')
ej_2c = ej_2c.select("DAY_OF_WEEK",
                    "AIRLINE",
                    "nombre",
                    ej_2c["avg(ARRIVAL_DELAY)"].alias("retraso"))

ej_2c.show()
ej_2c.repartition(1).write.csv('2c')

+-----------+-------+--------------+-------------------+
|DAY_OF_WEEK|AIRLINE|        nombre|            retraso|
+-----------+-------+--------------+-------------------+
|          1|     VX|Virgin America|-14.222222222222221|
|          6|     VX|Virgin America|            -11.125|
|          5|     VX|Virgin America|             -10.25|
+-----------+-------+--------------+-------------------+



### Ejercicio 2d

2d. ¿Cuál es el aeropuerto con mayor tráfico de entrada?

In [132]:
destino = flights.groupBy("DESTINATION_AIRPORT").count().sort(desc("count")).limit(3)
aeropuerto = airports.select("IATA_CODE", "AIRPORT")

ej_2d = destino.join(aeropuerto, destino.DESTINATION_AIRPORT == aeropuerto.IATA_CODE, 'left')
ej_2d.show()
ej_2d.repartition(1).write.csv('2d')

+-------------------+------+---------+--------------------+
|DESTINATION_AIRPORT| count|IATA_CODE|             AIRPORT|
+-------------------+------+---------+--------------------+
|                ATL|346904|      ATL|Hartsfield-Jackso...|
|                ORD|285906|      ORD|Chicago O'Hare In...|
|                DFW|239582|      DFW|Dallas/Fort Worth...|
+-------------------+------+---------+--------------------+



### Ejercicio 2e

2e. ¿Cuál es la aerolínea con mayor retraso de salida por día de la semana? 

In [133]:
retraso = flights.select(flights.DAY_OF_WEEK, 
                         flights.AIRLINE,
                         flights.DEPARTURE_DELAY.cast("float").alias("DEPARTURE_DELAY"))
retraso = retraso.groupBy("DAY_OF_WEEK", "AIRLINE").avg("DEPARTURE_DELAY")

retraso_2 = retraso.groupBy("DAY_OF_WEEK").max("avg(DEPARTURE_DELAY)")
retraso_2 = retraso_2.select(retraso_2.DAY_OF_WEEK.alias("dia_semana"),
                            retraso_2["max(avg(DEPARTURE_DELAY))"])
retraso_2 = retraso_2.join(retraso, 
               retraso_2["max(avg(DEPARTURE_DELAY))"] == retraso["avg(DEPARTURE_DELAY)"],
              'left')

ej_2e = retraso_2.select("DAY_OF_WEEK", "AIRLINE", "avg(DEPARTURE_DELAY)")
ej_2e.show()
ej_2e.repartition(1).write.csv('2e')

+-----------+-------+--------------------+
|DAY_OF_WEEK|AIRLINE|avg(DEPARTURE_DELAY)|
+-----------+-------+--------------------+
|          3|     UA|  14.282200834726739|
|          6|     NK|  15.780295269659536|
|          7|     NK|  17.149202320522118|
|          1|     NK|  18.795626679697044|
|          5|     NK|  15.786104837751099|
|          4|     UA|   14.88742760925581|
|          2|     NK|  15.714659197012137|
+-----------+-------+--------------------+



### Ejercicio 2f

2f. ¿Cuál es la tercer aerolínea con menor retraso de salida los lunes (day of week = 2)?

In [134]:
retraso = flights.select(flights.DAY_OF_WEEK, 
                         flights.AIRLINE,
                         flights.DEPARTURE_DELAY.cast("float").alias("DEPARTURE_DELAY"))

ej_2f = retraso.groupBy("DAY_OF_WEEK", "AIRLINE").avg("DEPARTURE_DELAY") \
.where("DAY_OF_WEEK = 2").sort(asc("avg(DEPARTURE_DELAY)")).limit(5)
ej_2f.show()
ej_2f.repartition(1).write.csv('2f')

+-----------+-------+--------------------+
|DAY_OF_WEEK|AIRLINE|avg(DEPARTURE_DELAY)|
+-----------+-------+--------------------+
|          2|     HA|  -1.554867003527505|
|          2|     AS|  0.7757525430766037|
|          2|     US|   6.475248598806726|
|          2|     AA|   6.750830434952519|
|          2|     VX|   7.467288669064748|
+-----------+-------+--------------------+



### Ejercicio 2g

2g. ¿Cuál es el aeropuerto origen que llega a la mayor cantidad de aeropuertos destino diferentes?

In [135]:
destino = flights.groupBy("ORIGIN_AIRPORT").agg(countDistinct("DESTINATION_AIRPORT"))
destino = destino.select("ORIGIN_AIRPORT", 
               destino["count(DISTINCT DESTINATION_AIRPORT)"].alias("destino"))
destino = destino.sort(desc("destino")).limit(3)
aeropuerto = airports.select("IATA_CODE", "AIRPORT")
ej_2g = destino.join(aeropuerto, destino.ORIGIN_AIRPORT == aeropuerto.IATA_CODE, "left")

ej_2g.show()
ej_2g.repartition(1).write.csv('2g')

+--------------+-------+---------+--------------------+
|ORIGIN_AIRPORT|destino|IATA_CODE|             AIRPORT|
+--------------+-------+---------+--------------------+
|           ATL|    169|      ATL|Hartsfield-Jackso...|
|           ORD|    162|      ORD|Chicago O'Hare In...|
|           DFW|    148|      DFW|Dallas/Fort Worth...|
+--------------+-------+---------+--------------------+

